<a href="https://colab.research.google.com/github/lauren-safwat/World-University-Rankings-Dashboard/blob/main/World_University_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip3 install dash
# !pip3 install jupyter_dash
# !pip3 install dash_bootstrap_components
# !pip install dash-trich-components

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import jupyter_dash
from dash import Dash, html, dcc, no_update
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
import dash_trich_components as dtc

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from math import ceil
from itertools import cycle

In [3]:
# import shutil
# shutil.rmtree('/content/World-University-Rankings-Dashboard')

In [4]:
# !git clone https://github.com/lauren-safwat/World-University-Rankings-Dashboard

In [5]:
%cd /content/World-University-Rankings-Dashboard
unis = pd.read_csv('dataset/World_University_Rankings.csv')

/content/World-University-Rankings-Dashboard


In [6]:
unis.head(10)

,University,Country,City,Region,Logo,Link,Type,Size,Faculty_count,International_Students,Students_count,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Latitude,Longitude
0,University Paris-Saclay,France,Paris,Europe,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/u...,Public,XL,8010,7649,40050,241,0,0,0,0,0,48.713651,2.203623
1,Baku Engineering University,Azerbaijan,Baku,Asia,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/b...,Private,S,225,78,4725,703,0,0,0,0,0,40.473905,49.727411
2,"Ecole normale supérieure, Paris",France,Paris,Europe,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/e...,Public,S,178,374,1958,33,43,0,0,0,0,48.841707,2.344792
3,Université Pierre et Marie Curie (UPMC),France,Paris,Europe,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/u...,Public,XL,4680,8782,42120,141,131,0,0,0,0,48.847301,2.357596
4,Université Paris-Sorbonne (Paris IV),France,Paris,Europe,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/u...,Public,XL,4680,8782,42120,221,293,0,0,0,0,48.849412,2.344538
5,"Université Paris-Dauphine, Université PSL",France,Paris,Europe,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/u...,Public,M,697,2039,8364,356,355,0,0,0,0,48.872848,2.273728
6,Arabian Gulf University,Bahrain,Manama,Asia,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/a...,Public,S,195,1011,1365,462,412,0,0,0,0,26.215342,50.571320
7,Khazar University,Azerbaijan,Baku,Asia,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/k...,Private,S,251,363,2510,703,700,0,0,0,0,40.408159,49.937186
8,Vasyl` Stus Donetsk National University,Ukraine,Vinnytsia,Europe,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/v...,Public,S,403,83,3627,703,800,0,0,0,0,49.225301,28.427554
9,Université Charles-de-Gaulle Lille 3,France,NaN,Europe,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/u...,Public,L,823,1179,18106,703,800,0,0,0,0,50.628579,3.126588


In [7]:
unis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1373 entries, 0 to 1372
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   University              1373 non-null   object 
 1   Country                 1373 non-null   object 
 2   City                    1322 non-null   object 
 3   Region                  1373 non-null   object 
 4   Logo                    1373 non-null   object 
 5   Link                    1373 non-null   object 
 6   Type                    1373 non-null   object 
 7   Size                    1373 non-null   object 
 8   Faculty_count           1373 non-null   int64  
 9   International_Students  1373 non-null   int64  
 10  Students_count          1373 non-null   int64  
 11  Rank_2017               1373 non-null   int64  
 12  Rank_2018               1373 non-null   int64  
 13  Rank_2019               1373 non-null   int64  
 14  Rank_2020               1373 non-null   

In [8]:
app = jupyter_dash.JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP, 'assets/css/style.css'], suppress_callback_exceptions=True)

# **Distribution of universities across the world page**

In [9]:
world = dbc.Container([
    dbc.Row(html.H1('🎓 Distribution of Universities Across the World'),id='page_title'),
    dbc.Row([
        dbc.Col(dcc.Graph(id='map', figure={}), width=9),
        dbc.Col(html.Div(id='table', style={'maxHeight': '410px', 'overflowY': 'scroll'}),width=3)
    ], id='map_table'),

    dbc.Row([
        dbc.Col(dcc.Slider(id='mapSlider',
                  min=2017,
                  max=2022,
                  value=2022,
                  step=None,
                  marks={i: str(i) for i in range(2017, 2023)}
                  ), width=6),
             
        dbc.Col(dcc.Dropdown([str(region) for region in unis['Region'].unique()], placeholder='Select Region', id='map_region'), width=2),
        dbc.Col(dcc.Dropdown(placeholder='Select Country', id='map_country'),width=2),
        dbc.Col(dcc.Checklist([
            {'label': ' Private Universities', 'value': 'Private'},
            {'label': ' Public Universities', 'value': 'Public'},
        ],
        id='checklist'
      ),width=2)
    ],id='controls_row'),
    

    dbc.Row([
        dbc.Col(
            [
             dbc.Row(dcc.Graph(id='bar_uniCount', figure={})),],
            width=4
        ),

        dbc.Col(
            dbc.Row(dcc.Graph(id='bar_intCount', figure={})),
            width=4
        ),
        dbc.Col(
            dbc.Row(dcc.Graph(id='pie_type', figure={})),
            width=4
        )
      ]),
    
    ])

In [10]:
@app.callback(
    Output('map_country', 'options'),
    Output('map', 'figure'),
    Output("table", "children"),
    Input('mapSlider', 'value'),
    Input('map_region', 'value'),
    Input('map_country', 'value'),
    Input('checklist', 'value')
)


def updateMap(year, region, country,priv):
    year = 'Rank_' + str(year)
    df = unis[unis[year] > 0].sort_values(year, axis=0)
    print
    zoom = 0.1

    if region:
        zoom = 1.3
        df = df[df['Region'] == region]

    if country:
        zoom = 2.5
        df = df[df['Country'] == country]
    if(priv):
      df=df[df['Type'].isin(priv)]

    
    fig = px.scatter_mapbox(df[:100],
                            lon='Longitude',
                            lat='Latitude',
                            color=year,
                            hover_name='University',
                            hover_data={'Latitude':False, 'Longitude':False, 'Country':True,'City':True},
                            zoom=zoom,
                            mapbox_style='carto-positron',
                            color_continuous_scale=px.colors.sequential.deep_r,
                            )
    
    fig.update_traces(marker = {'size':10, 'opacity':0.5}, selector={'type': 'scattermapbox'})

    countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]

    table = dbc.Table.from_dataframe(df[[year, 'University']], striped=True, bordered=True, hover=True, responsive=True)
    
    return countries, fig, table

In [11]:
@app.callback(
    Output('bar_uniCount', 'figure'),
    Input('map_region', 'value'),
    Input('checklist', 'value')
)

def updateBar1(region, priv):
    title = 'University Count in the World'
    x=unis.groupby(['Region'])['University'].count()
    y=x.keys()
    text1 = 'Region'    
    df=unis
    if(priv):
      df=df[df['Type'].isin(priv)]
      x=df.groupby(['Region'])['University'].count()
      y=x.keys()
    if region:
        title = 'University Count in '+str(region)
        df = df[df['Region'] == region]
        x=df.groupby(['Country'])['University'].count()
        y=x.keys()
        text1 = 'Country'
    sortd = sorted(zip(x,y))
    x = [i[0] for i in sortd]
    y = [i[1] for i in sortd]
        
    fig = go.Figure(go.Bar(
                x=list(x),
                y=list(y),
                orientation='h',
                text=x,
                hovertemplate = "<br>"+text1+": %{y} </br> Count:%{text}<extra></extra>",
                marker={
                'color': x,
                'colorscale': 'bluyl'
                }))
    fig.update_layout(title=title,
                      barmode='group',
                      bargap=0.0,
                      bargroupgap=0.0
                     )
    return fig

In [12]:
@app.callback(
    Output('bar_intCount', 'figure'),
    Input('map_region', 'value'),
    Input('checklist', 'value')
)

def updateBar2(region,priv):
    title = 'International Students in the World'
    x=unis.groupby(['Region'])['International_Students'].sum()
    y=x.keys()
    text1 = 'Region'    
    df=unis
    if(priv):
      df=df[df['Type'].isin(priv)]
      x=df.groupby(['Region'])['International_Students'].sum()
      y=x.keys()

    if region:
        title = 'International Students in '+str(region)
        text1 = 'Country'
        df = df[df['Region'] == region]
        x=df.groupby(['Country'])['International_Students'].sum()
        y=x.keys()
    sortd = sorted(zip(x,y))
    x = [i[0] for i in sortd]
    y = [i[1] for i in sortd]
    
    fig = go.Figure(go.Bar(
                x=list(x),
                y=list(y),
                orientation='h',
                text=x,
                hovertemplate = "<br>"+text1+": %{y} </br> Count:%{text}<extra></extra>",
                marker={
                'color': x,
                'colorscale': 'bluyl'
                }))
    fig.update_layout(title=title,
                      barmode='group',
                      bargap=0.0,
                      bargroupgap=0.0)
    return fig

In [13]:
@app.callback(
    Output('pie_type', 'figure'),
    Input('map_region', 'value'),
    Input('map_country', 'value')
)

def updatePie(region,country):
    title = 'Public VS Private Universities in the World'
    df=unis
    labels=['Private','Public']  
    text = 'World'      

    if region:
        title = 'Public VS Private Universities in '+region
        df = unis[unis['Region'] == region]
        text = region  
    if country :
      title = 'Public VS Private Universities in '+country
      df = df[df['Country'] == country]
      text=country
    
    values=[df[df['Type']=='Private'].count()['University'],df[df['Type']=='Public'].count()['University']]
    countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values,
                             text=values,
                             hole=.3,
                             marker = dict(colors = ['rgb(148,212,180)','rgb(4,82,117,255)']),
                             textinfo='label+percent',
                             insidetextorientation='radial',
                             hovertemplate = "<br>Type: %{label} </br> Count:%{value}<extra></extra>",
                             )])
    fig.update_layout(title=title)
    fig.add_annotation(dict(font=dict(color='black',size=15),
                                        x=0,
                                        y=-0.12,
                                        showarrow=False,
                                        text=text,
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
    return fig


# **Universities page**

In [14]:
uni = dbc.Container([
      dbc.Row([
          dbc.Col(html.H1('👩🏻‍🎓 Ranking of Top 5 Universities Over the World', id='page2_title'))
      ]),

      dbc.Row([
          dbc.Col(dcc.Dropdown([str(region) for region in unis['Region'].unique()], placeholder='Select Region',multi=True, id='line_region_uni'),width=4),
          dbc.Col(dcc.Dropdown(placeholder='Select Country',multi=True, id='line_country_uni'),width=4),
          dbc.Col(dcc.Dropdown(placeholder='Select University',multi=True, id='line_uni_uni'),width=4)
      ], className='dropDowns'),  

      dbc.Row([
          html.H3('Ranking of Universites over the Last 6 Years'),
          dcc.Graph(id='line_uni', figure={})
      ]),

      dbc.Row([
          dbc.Col(dtc.Carousel([],
              id='carousel',
              slides_to_scroll=1,
              swipe_to_slide=True,
              autoplay=False,
              arrows=True
          ))
      ])
    
])

In [15]:
@app.callback(
    Output('line_uni_uni', 'options'),
    Output('line_country_uni', 'options'),
    Output('line_uni', 'figure'),
    Input('line_region_uni', 'value'),
    Input('line_country_uni', 'value'),
    Input('line_uni_uni', 'value')
)

def updateLine(region, country, university):
    df = unis
    
    palette = cycle(px.colors.cyclical.Phase)
    palette2=cycle(px.colors.cyclical.Phase)

    if region :
        df = unis[unis['Region'].isin(region) ]
    if country :
        df = df[df['Country'].isin(country)]
    if not university:
        university = list(df[df['Rank_2022']>0].iloc[0:5, 0].values)
    fig = go.Figure()
    for uni in university:
      x=np.arange(2017, 2023)
      y=df[df['University']==uni].iloc[:,11:17].values.flatten().tolist()
      x = [x[i] for i in range(len(y)) if y[i]>0]
      y = [y[i] for i in range(len(y)) if y[i]>0]
      
      fig.add_trace(go.Scatter(x=x, y=y, name=uni,
                          line_shape='linear',
                          line_color=next(palette),
                          hovertemplate = "<br>Year : %{x} </br> Rank:%{y}<extra></extra>"))
      fig.add_trace(go.Scatter(x=[x[y.index(np.min(y))]], y=[np.min(y)],name=str(min(y)),
                              mode = 'markers',
                              marker_symbol = 'circle',
                              marker_color=next(palette2),
                              marker_size = 10,
                              hovertemplate = "<br>Year : %{x} </br> Rank:%{y}<extra></extra>",
                              showlegend=False))

    fig.update_layout(
        yaxis=dict(
            autorange='reversed',
        ),
        xaxis=dict(
            dtick=1,
            range=[2016.1, 2022.9]
        ),
      legend=dict(
          title_font_family="Times New Roman",
          font=dict(
              family="Courier",
              size=12,
              color="black"
          ),
        
      )
  )
    
    countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]
    universities = [{'label': str(i), 'value':str(i)} for i in df['University'].unique()]
    return universities, countries, fig



In [16]:
@app.callback(
    Output('carousel', 'children'),
    Input('line_region_uni', 'value'),
    Input('line_country_uni', 'value'),
    Input('line_uni_uni', 'value')
)

def updateCards(region, country, university):
    df = unis

    if region :
        df = df[df['Region'].isin(region)]
    if country :
        df = df[df['Country'].isin(country)]

    universities = df[df['Rank_2022']>0][0:5]

    if university:  
        universities = df[df['University'].isin(university)]

    labels=['International Students', 'National Students']

    cards = []

    for i in range(universities.shape[0]):

        fig = go.Figure()
        international_students = universities.iloc[i, 9]
        students = universities.iloc[i, 10]
        values = [international_students, students-international_students]

        fig.add_trace(go.Pie(labels=labels, values=values,
                                text=values,
                                hole=.3,
                                  marker = dict(colors = ['rgb(148,212,180)','rgb(4,82,117,255)']),
                                textinfo='percent',
                                insidetextorientation='radial',
                                hovertemplate = "<br>%{label} </br> Count:%{value}<extra></extra>",)
        )

        fig.update_layout(margin=dict(t=0, b=0, l=0, r=0))

        uni_data = []
        if universities.iloc[i, 2]:
            uni_data.append(html.Li('City: ' + universities.iloc[i, 2]))
        uni_data.append(html.Li('Country: ' + universities.iloc[i, 1]))
        uni_data.append(html.Li('Region: ' + universities.iloc[i, 3]))
        uni_data.append(html.Li('Type: ' + universities.iloc[i, 6] + ' University'))

        card_content = [
          html.A([
              dbc.CardHeader([
                 # html.Img(src = universities.iloc[i, 4]),
                  html.H5(universities.iloc[i, 0])
              ], id='card_header')
          ], href=universities.iloc[i, 5], target='_blank'),

          dbc.CardBody([
              html.Ul(uni_data, id='uni_data'),
              dcc.Graph(figure = fig, style={"height": "70%", "width": "100%"})
          ]),
        ]
        cards.append(dbc.Card(card_content, outline=True,id='uni_card'))
    
    return cards



# **Dashboard Layout**

In [17]:
app.layout = html.Div(children=[
     dbc.Navbar(children=[
        dbc.Col([
            html.Img(id='logo', src=app.get_asset_url('images/World2.png')),
            html.H1('World University Rankings', id='title')
        ], width=8),

        dbc.Col(dbc.Tabs(id='tabs', active_tab='tab-1', children=[
            dbc.Tab(label="World Overview", tab_id='tab-1'),
            dbc.Tab(label="Universities", tab_id='tab-2')
        ]))
     ], id='nav', sticky = 'top'),
   
     html.Div(id='content')
], className='dashboard')

In [18]:
@app.callback(
    Output('content', 'children'),
    Input('tabs', 'active_tab'),
)

def display_content(active_tab):
    if active_tab == 'tab-1':
        return world
    return uni

In [19]:
app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>